In [ ]:
# STEP 1: Import libraries
import pandas as pd
import folium
from folium.plugins import MarkerCluster
import numpy as np
import math
from geopy.distance import geodesic

# STEP 2: Load Excel file
file_path = '/content/drive/MyDrive/EV Project Davise Lab/ev_final.xlsx'  # Update path if needed
df = pd.read_excel(file_path)

# STEP 3: Drop missing coordinates and filter by city and address
lat_col = 'latitude'
lon_col = 'longitude'
name_col = 'Station Name' if 'Station Name' in df.columns else 'name'

# Remove Secunderabad and keep only Delhi stations
df = df[~df['address'].str.contains('Secunderabad', na=False)]
df = df[df['city'].isin(['Delhi', 'New Delhi'])]
df = df.dropna(subset=[lat_col, lon_col])
print(f"🔢 Valid stations before deduplication: {len(df)}")

# STEP 4: Remove duplicate coordinates
if name_col:
    df = df.drop_duplicates(subset=[lat_col, lon_col, name_col])
else:
    df = df.drop_duplicates(subset=[lat_col, lon_col])
print(f"✅ After removing duplicates: {len(df)}")

# STEP 5: Define grid bounds and size
lat_min, lat_max = 28.40, 28.88
lon_min, lon_max = 76.84, 77.33
grid_size_main = 0.008
grid_size_sub = 0.004

# STEP 6: Assign main zones
df['Grid_Row'] = ((df[lat_col] - lat_min) // grid_size_main).astype(int)
df['Grid_Col'] = ((df[lon_col] - lon_min) // grid_size_main).astype(int)
df['Zone_ID'] = df['Grid_Row'].astype(str) + '_' + df['Grid_Col'].astype(str)

# STEP 7: Count and merge zone station counts
zone_counts = df['Zone_ID'].value_counts().reset_index()
zone_counts.columns = ['Zone_ID', 'Station_Count']
df = df.merge(zone_counts, on='Zone_ID', how='left')

# STEP 8: Subdivide overcrowded zones
overcrowded = zone_counts[zone_counts['Station_Count'] > 10]['Zone_ID'].tolist()

def assign_subzone(row):
    if row['Zone_ID'] not in overcrowded:
        return row['Zone_ID']
    lat0 = lat_min + row['Grid_Row'] * grid_size_main
    lon0 = lon_min + row['Grid_Col'] * grid_size_main
    row_sub = int((row[lat_col] - lat0) // grid_size_sub)
    col_sub = int((row[lon_col] - lon0) // grid_size_sub)
    return f"{row['Zone_ID']}_{row_sub}_{col_sub}"

df['Zone_ID_Final'] = df.apply(assign_subzone, axis=1)

# STEP 9: Remove malformed zone IDs
df = df[~df['Zone_ID_Final'].str.contains('-')]

# STEP 10: Final zone station counts
final_zone_counts = df['Zone_ID_Final'].value_counts().reset_index()
final_zone_counts.columns = ['Zone_ID_Final', 'Station_Count']
df = df.merge(final_zone_counts, on='Zone_ID_Final', how='left')

print(f"📊 Final number of zones: {df['Zone_ID_Final'].nunique()}")

# STEP 11: Estimate zone areas
zone_area_main = grid_size_main * 111 * grid_size_main * 111 * math.cos(math.radians(28.6))
zone_area_sub = grid_size_sub * 111 * grid_size_sub * 111 * math.cos(math.radians(28.6))
print(f"📐 Main zone area: ~{zone_area_main:.3f} km²")
print(f"📐 Subdivided zone area: ~{zone_area_sub:.3f} km²")

# STEP 12: Create Folium map
m = folium.Map(location=[28.6139, 77.2090], zoom_start=11)
marker_cluster = MarkerCluster().add_to(m)

# STEP 13: Add zones to map
def get_color(count):
    if count <= 3:
        return 'green'
    elif count <= 10:
        return 'orange'
    else:
        return 'red'

drawn = set()
for _, row in df.iterrows():
    zid = row['Zone_ID_Final']
    if zid in drawn:
        continue

    parts = zid.split('_')
    if len(parts) == 2:
        r, c = int(parts[0]), int(parts[1])
        lat0 = lat_min + r * grid_size_main
        lon0 = lon_min + c * grid_size_main
        size = grid_size_main
    elif len(parts) == 4:
        r, c, sr, sc = map(int, parts)
        lat0 = lat_min + r * grid_size_main + sr * grid_size_sub
        lon0 = lon_min + c * grid_size_main + sc * grid_size_sub
        size = grid_size_sub
    else:
        continue

    count = row['Station_Count_y']
    color = get_color(count)
    square = [
        [lat0, lon0],
        [lat0, lon0 + size],
        [lat0 + size, lon0 + size],
        [lat0 + size, lon0],
        [lat0, lon0]
    ]
    folium.Polygon(
        square,
        color=color,
        weight=1,
        fill=True,
        fill_opacity=0.4,
        popup=f"Zone: {zid}<br>Stations: {count}"
    ).add_to(m)
    drawn.add(zid)

# STEP 14: Add station markers
for _, row in df.iterrows():
    folium.Marker(
        location=[row[lat_col], row[lon_col]],
        popup=f"{row.get(name_col, 'Charging Station')}<br>Zone: {row['Zone_ID_Final']}<br>Count: {row['Station_Count_y']}"
    ).add_to(marker_cluster)

# STEP 15: Define vehicle assignment function
def assign_vehicle_to_station(vehicle_coords, df):
    df['distance_km'] = df.apply(
        lambda row: geodesic(vehicle_coords, (row[lat_col], row[lon_col])).km,
        axis=1
    )
    nearest = df.sort_values('distance_km').iloc[0]
    return {
        'Station Name': nearest.get(name_col, 'Unknown'),
        'Address': nearest['address'],
        'Distance (km)': round(nearest['distance_km'], 3),
        'Zone_ID_Final': nearest['Zone_ID_Final'],
        'Latitude': nearest[lat_col],
        'Longitude': nearest[lon_col]
    }

# STEP 16: Sample vehicle coordinates
vehicle_coords = (28.7380	77.1273)  # Replace with your own coordinates  28.5600, 77.1000 28.6667, 77.1200 28.622345, 77.197843 28.7333, 77.1333 28.7380	77.1273
result = assign_vehicle_to_station(vehicle_coords, df)

# STEP 17: Add vehicle + nearest station to map
folium.Marker(
    location=vehicle_coords,
    popup='🚗 Vehicle Location',
    icon=folium.Icon(color='blue', icon='car', prefix='fa')
).add_to(m)

folium.Marker(
    location=[result['Latitude'], result['Longitude']],
    popup=f"🔋 Nearest Station:<br>{result['Station Name']}",
    icon=folium.Icon(color='red', icon='bolt', prefix='fa')
).add_to(m)

# STEP 18: Display result and map
print("🚘 Nearest Charging Station Assigned:")
for k, v in result.items():
    print(f"{k}: {v}")




🔢 Valid stations before deduplication: 2313
✅ After removing duplicates: 1899
📊 Final number of zones: 663
📐 Main zone area: ~0.692 km²
📐 Subdivided zone area: ~0.173 km²
🚘 Nearest Charging Station Assigned:
Station Name: BluSmart
Address: Multi Level Car Parking, Level 2 , Indira Gandhi International Airport , New Delhi
Distance (km): 1.283
Zone_ID_Final: 19_31
Latitude: 28.55476
Longitude: 77.08831
